<style type="text/css">.tg  {border-collapse:collapse;border-spacing:0;}.tg td{border-color:rgb(16, 137, 182);border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}.tg th{border-color:rgb(16, 137, 182);border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}.tg .tg-73oq{border-color:rgb(10, 89, 162);text-align:left;vertical-align:middle}.tg .tg-42lt{border-color:rgb(10, 89, 162);text-align:center;vertical-align:middle}.tg .tg-5qt9{font-size:small;text-align:left;vertical-align:middle}</style><table class="tg"><thead>  <tr>    <th class="tg-73oq"><img src="https://raw.githubusercontent.com/euroargodev/argopy/master/docs/_static/argopy_logo_long.png" alt="Argopy logo" width="120" height="60"></th>    <th class='tg-42lt'><h1>JSON web-API requests handling</h1></th>  </tr></thead><tbody>  <tr>    <td class="tg-5qt9" colspan="2"><span style="font-weight:bold">Author :</span> <a href="//annuaire.ifremer.fr/cv/17182" target="_blank" rel="noopener noreferrer">G. Maze</a></td>  </tr>  <tr>    <td class="tg-5qt9" colspan="2">🏷️ This notebook is compatible with Argopy versions &gt;= <a href="https://argopy.readthedocs.io/en/v1.3.1" target="_blank" rel="noopener noreferrer">v1.3.1</a></td>  </tr>  <tr>    <td class="tg-5qt9" colspan="2">© <a href="https://github.com/euroargodev/argopy-training/blob/main/LICENSE" target="_blank" rel="noopener noreferrer">European Union Public Licence (EUPL) v1.2</a>, see at the bottom of this notebook for more.</td>  </tr></tbody></table>
**Description:**

This notebook describes how to use the internal [http file system (↗)](https://argopy.readthedocs.io/en/v1.3.1/generated/argopy.stores.httpstore.html) to handle multiple requests to a web-API and to process JSON responses.

This is an illustration show-casing this *private* feature for those who would be interested in contributing to **Argopy**.


**Table of Contents**
  - [Creating a file system for http requests](#creating-a-file-system-for-http-requests)
  - [Load JSON from one request](#load-json-from-one-request)
    - [✏️ EXERCISE](#✏️-exercise)
  - [Load JSON from multiple requests](#load-json-from-multiple-requests)
  - [Preprocess JSON from multiple requests](#preprocess-json-from-multiple-requests)
    - [✏️ EXERCISE](#✏️-exercise)
    - [✏️ EXERCISE](#✏️-exercise)
***

Let's start with the filesystem import:

In [ ]:
# Shut down some warning messages for clarity
import warnings
warnings.filterwarnings("ignore")

# Import the Argopy features we want to work with in this notebook:
from argopy.stores import httpstore

### Creating a file system for http requests

To create a [httpstore (↗)](https://argopy.readthedocs.io/en/v1.3.1/generated/argopy.stores.httpstore.html) instance, there is no need for arguments.

By default, request responses are not cached, but here we'll use cache for demo:

In [ ]:
fs = httpstore(cache=True)
fs

### Load JSON from one request

Here, we'll use the [open_json (↗)](https://argopy.readthedocs.io/en/v1.3.1/generated/argopy.stores.httpstore.open_json.html) method.

For this demo, we'll send requests to the [Euro-Argo fleet-monitoring web-API (↗)](https://fleetmonitoring.euro-argo.eu/swagger-ui.html): https://fleetmonitoring.euro-argo.eu:

In [ ]:
api = "https://fleetmonitoring.euro-argo.eu"

<br>

Let's first request basic meta-data for a given float:

In [ ]:
WMO = 6903091
uri = f"{api}/floats/basic/{WMO}"
data = fs.open_json(uri)
data.keys()

<br>

Obviously, you can do this in oneliner:

In [ ]:
data = httpstore().open_json('https://fleetmonitoring.euro-argo.eu/floats/basic/6903091')
data.keys()

#### ✏️ EXERCISE

Use the Euro-Argo dataselection web-API to retrieve the trajectory of a given float

💡 Code hint:

The endpoint is:
```
api = "https://dataselection.euro-argo.eu/api"
uri = f"{api}/trajectory/{WMO}"
```

In [ ]:
# Your code

### Load JSON from multiple requests

With the [open_mfjson (↗)](https://argopy.readthedocs.io/en/v1.3.1/generated/argopy.stores.httpstore.open_mfjson.html) method, it is easy to retrieve many JSON data (from multiple http requests) in parallel.

Let's re-use the EA Data-selection web-API from above, to retrieve the trajectory of several floats.

First we create a store:

In [ ]:
fs = httpstore(cache=True)
fs

<br>

then we compute the list of web-API requests to execute:

In [ ]:
WMO = [6902771, 6902772]
api = "https://dataselection.euro-argo.eu/api"
uris = [f"{api}/trajectory/{wmo}" for wmo in WMO]
uris

<br>

and we can retrieve all URIs in a single line. We get the results as a list of JSON data.

🛟 NOTE    
Since all requests may be executed in parallel, asynchroneously, there is no guarantee than the list of results to be ordered as the list URIs.

In [ ]:
data_list = fs.open_mfjson(uris)
assert len(data_list) == len(uris)

In [ ]:
for chunk in data_list:
    print(f"- {len(chunk)} profiles for this float")

In [ ]:
data_list[0][0]

### Preprocess JSON from multiple requests

We will now show how to process each JSON requests upon reception from the client using the ``preprocess`` argument.

Let's first define a processing function that will receive a JSON dictionary and return something else:

In [ ]:
def jsprocessor(data):
    """
    data is a list of dict. like:
    {'id': 4452726,
     'cvNumber': 60,
     'coordinate': {'lat': 27.51028, 'lon': -63.75963},
     'level': 0}

    In this function we transform this list into a more succint list of dict. like:
    {'lat': 27.51028, 'lon': -63.75963, 'cyc': 60, 'id': 4452726}
    """
    result = []
    for profile in data:
        coords = profile['coordinate']
        coords.update({'cyc': int(profile['cvNumber']), 'id': profile['id']})
        result.append(coords)
    return result

<br> 

Let's ensure it works as expected:

In [ ]:
jsprocessor(data_list[0])[0:3]

<br>

In order to process all requests this way, we can do:

In [ ]:
data_list = fs.open_mfjson(uris, preprocess=jsprocessor)
assert len(data_list) == len(uris)

In [ ]:
data_list[0][0:3]

#### ✏️ EXERCISE

Modify the processor above to return coordinates for a range of cycle number, not all cycles.

💡 Code hint:  
Passing arguments to the json processor is done with the ``preprocess_opts`` argument of ``open_mfjson``.

In [ ]:
# Your code

#### ✏️ EXERCISE

Again, modify the processor above to add the float WMO to the output list of dictionary.

💡 Code hint:  
One can pass the url of the web-API requests down to the json processor with the ``follow_url`` boolean argument of ``open_mfjson``.

In [ ]:
# Your code


## 🏁 End of the notebook

***
#### 👀 Useful argopy commands
```python
argopy.reset_options()
argopy.show_options()
argopy.status()
argopy.clear_cache()
argopy.show_versions()
```
#### ⚖️ License Information
This Jupyter Notebook is licensed under the **European Union Public Licence (EUPL) v1.2**.

| Permissions      | Limitations     | Conditions                     |
|------------------|-----------------|--------------------------------|
| ✔ Commercial use | ❌ Liability     | ⓘ License and copyright notice |
| ✔ Modification   | ❌ Trademark use | ⓘ Disclose source              |
| ✔ Distribution   | ❌ Warranty      | ⓘ State changes                |
| ✔ Patent use     |                  | ⓘ Network use is distribution  |
| ✔ Private use    |                  | ⓘ Same license                 |

For more details, visit: [EUPL v1.2 Full Text (↗)](https://github.com/euroargodev/argopy-training/blob/main/LICENSE).

#### 🤝 Sponsor
![logo (↗)](https://raw.githubusercontent.com/euroargodev/argopy-training/refs/heads/main/for_nb_producers/disclaimer_argopy_EAONE.png)
***
